In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import datetime

In [2]:
from pypfopt import expected_returns
from pypfopt import risk_models
from pypfopt import efficient_frontier
from pypfopt import objective_functions

In [21]:
import pandas as pd
import pandas_datareader as web
import datetime

def _load_prices(
    ticker: str,
    data_source: str = 'yahoo',
    end: str = datetime.date.today()
) -> pd.DataFrame:
    return web.DataReader(name = ticker, data_source = data_source, start = '2000-01-01', end = end).loc[:, 'Adj Close']

def load_prices_multitickers(
    tickers: list,
    data_source: str = 'yahoo',
    end: str = datetime.date.today()
) -> list:
    prices = pd.DataFrame()
    for t in tickers:
        prices[t] = _load_prices(t, data_source, end)
        
    return prices.fillna(value=None, method="ffill", axis=0, inplace=False).dropna(axis=0, how='any', inplace=False)

In [22]:
tickers = ['VOO', 'BND', 'VXUS', 'BNDX']
load_prices_multitickers(tickers)

,VOO,BND,VXUS,BNDX
Date,,,,
2013-06-04,127.572510,65.388733,37.280701,41.435829
2013-06-05,125.616463,65.364861,36.601757,41.485256
2013-06-06,126.602997,65.452423,36.825493,41.427586
2013-06-07,128.372009,65.285202,37.049236,41.345177
2013-06-10,128.320984,65.117989,37.018375,41.048660
...,...,...,...,...
2022-02-18,399.290009,81.529999,61.540001,53.590000
2022-02-22,394.920013,81.480003,60.810001,53.360001
2022-02-23,387.809998,81.070000,60.209999,53.340000


In [3]:
voo = web.DataReader(name='VOO', 
    data_source='yahoo', 
    start='2000-01-01',
    end=datetime.date.today()
)
voo

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-09-09,102.500000,101.139999,102.500000,101.320000,26500.0,81.350792
2010-09-10,101.860001,101.300003,101.680000,101.779999,8600.0,81.720116
2010-09-13,103.139999,102.500000,102.959999,103.059998,33750.0,82.747864
2010-09-14,103.480003,102.379997,102.839996,103.040001,59400.0,82.731812
2010-09-15,103.379997,102.400002,102.620003,103.300003,9250.0,82.940567
...,...,...,...,...,...,...
2022-02-16,411.890015,406.320007,408.200012,410.589996,7743800.0,410.589996
2022-02-17,407.980011,401.230011,407.489990,401.859985,7541000.0,401.859985
2022-02-18,403.309998,397.000000,402.149994,399.290009,10565400.0,399.290009


In [4]:
bnd = web.DataReader(name='BND', 
    data_source='yahoo', 
    start='2000-01-01',
    end=datetime.date.today()
)
bnd

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2007-04-10,75.260002,75.000000,75.070000,75.239998,35000.0,47.704529
2007-04-11,75.290001,75.029999,75.160004,75.040001,87700.0,47.577717
2007-04-12,75.080002,74.959999,75.059998,75.029999,78100.0,47.571365
2007-04-13,75.070000,74.849998,75.040001,74.910004,18000.0,47.495331
2007-04-16,74.989998,74.940002,74.989998,74.980003,52700.0,47.539680
...,...,...,...,...,...,...
2022-02-16,81.260002,80.959999,81.239998,81.209999,7142400.0,81.209999
2022-02-17,81.489998,81.250000,81.300003,81.389999,6833600.0,81.389999
2022-02-18,81.550003,81.410004,81.500000,81.529999,4744100.0,81.529999


In [5]:
vxus = web.DataReader(name='VXUS', 
    data_source='yahoo', 
    start='2000-01-01',
    end=datetime.date.today()
)
vxus

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2011-01-28,53.830002,49.259998,50.070000,49.330002,40800.0,35.691998
2011-01-31,50.000000,49.630001,49.750000,49.880001,46200.0,36.089951
2011-02-01,50.950001,50.150002,50.619999,50.950001,26300.0,36.864120
2011-02-02,51.000000,50.540001,51.000000,50.689999,47700.0,36.676003
2011-02-03,50.990002,50.360001,50.990002,50.779999,51100.0,36.741119
...,...,...,...,...,...,...
2022-02-16,62.970001,62.310001,62.320000,62.799999,4709900.0,62.799999
2022-02-17,62.480000,61.869999,62.439999,61.930000,4655500.0,61.930000
2022-02-18,61.990002,61.389999,61.840000,61.540001,5668900.0,61.540001


In [27]:
bndx = web.DataReader(name='BNDX', 
    data_source='yahoo', 
    start='2000-01-01',
    end=datetime.date.today()
)
bndx

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2013-06-04,50.599998,50.240002,50.599998,50.299999,44300.0,41.435822
2013-06-05,50.400002,50.299999,50.400002,50.360001,62500.0,41.485264
2013-06-06,50.349998,50.270000,50.340000,50.290001,106800.0,41.427582
2013-06-07,50.330002,50.189999,50.330002,50.189999,70800.0,41.345200
2013-06-10,50.259998,49.799999,50.259998,49.830002,128800.0,41.048656
...,...,...,...,...,...,...
2022-02-16,53.349998,53.240002,53.320000,53.330002,3462500.0,53.330002
2022-02-17,53.450001,53.310001,53.310001,53.419998,2509500.0,53.419998
2022-02-18,53.599998,53.490002,53.509998,53.590000,2146900.0,53.590000


In [28]:
prices = pd.DataFrame()
prices['VOO'] = voo['Adj Close']
prices['BND'] = bnd['Adj Close']
prices['VXUS'] = vxus['Adj Close']
prices['BNDX'] = bndx['Adj Close']
prices = prices.fillna(value=None, method="ffill", axis=0, inplace=False).dropna(axis=0, how='any', inplace=False)
prices

,VOO,BND,VXUS,BNDX
Date,,,,
2013-06-04,127.572533,65.388725,37.280704,41.435822
2013-06-05,125.616432,65.364845,36.601749,41.485264
2013-06-06,126.603004,65.452423,36.825489,41.427582
2013-06-07,128.372025,65.285194,37.049244,41.345200
2013-06-10,128.320953,65.118027,37.018375,41.048656
...,...,...,...,...
2022-02-16,410.589996,81.209999,62.799999,53.330002
2022-02-17,401.859985,81.389999,61.930000,53.419998
2022-02-18,399.290009,81.529999,61.540001,53.590000


In [29]:
msci = pd.read_csv("./data/msci.csv", sep=",", header=0, index_col="Date", skipinitialspace=True, skip_blank_lines=True, parse_dates=True, infer_datetime_format=True, thousands=",")
msci.iloc[:, 0]

Date
2007-11-30    1291.856
2007-12-31    1275.956
2008-01-31    1170.327
2008-02-29    1173.998
2008-03-31    1152.170
                ...   
2021-01-29    2094.703
2021-02-26    2149.293
2021-03-31    2200.156
2021-04-30    2292.451
2021-05-31    2322.886
Name: ACWI + FRONTIER MARKETS(ACWI FM) ALL CAP All Cap (Large+Mid+Small+Micro Cap), Length: 163, dtype: float64

In [30]:
mhr = expected_returns.mean_historical_return(prices, frequency=252)
mhr

VOO     0.136018
BND     0.024963
VXUS    0.056523
BNDX    0.029391
dtype: float64

In [33]:
ehr = expected_returns.ema_historical_return(prices, frequency=252, span=252*5)
ehr

VOO     0.161238
BND     0.010837
VXUS    0.081373
BNDX    0.008571
Name: 2022-02-23 00:00:00, dtype: float64

In [34]:
risk_free_rate=1.005**((1/252)) - 1
risk_free_rate

1.9792027252663047e-05

In [35]:
capmr = expected_returns.capm_return(prices, market_prices=None, risk_free_rate=risk_free_rate, compounding=True, frequency=252)
capmr

VOO     0.123673
BND     0.007918
VXUS    0.123490
BNDX    0.005168
Name: mkt, dtype: float64

In [36]:
sample_cov = risk_models.risk_matrix(prices, method='sample_cov')
sample_cov

,VOO,BND,VXUS,BNDX
VOO,0.028771,0.000164,0.024910,0.000184
BND,0.000164,0.001968,0.000443,0.000877
VXUS,0.024910,0.000443,0.028396,0.000201
BNDX,0.000184,0.000877,0.000201,0.000988


In [37]:
semicovariance = risk_models.risk_matrix(prices, method='semicovariance', benchmark=risk_free_rate, frequency=252, log_returns=False)
semicovariance

,VOO,BND,VXUS,BNDX
VOO,0.014933,0.001407,0.013733,0.000986
BND,0.001407,0.001073,0.001603,0.000564
VXUS,0.013733,0.001603,0.015871,0.001068
BNDX,0.000986,0.000564,0.001068,0.000545


In [38]:
exp_cov = risk_models.risk_matrix(prices, method='exp_cov', span=252*5, frequency=252)
exp_cov

,VOO,BND,VXUS,BNDX
VOO,0.038072,0.000918,0.031320,0.000641
BND,0.000918,0.002670,0.001033,0.001183
VXUS,0.031320,0.001033,0.033146,0.000624
BNDX,0.000641,0.001183,0.000624,0.001159


In [39]:
ledoiid_wolf_cc = risk_models.risk_matrix(prices, method='ledoit_wolf_constant_correlation', frequency=252)
ledoiid_wolf_cc

,VOO,BND,VXUS,BNDX
VOO,0.028771,0.000203,0.024559,0.000210
BND,0.000203,0.001968,0.000476,0.000866
VXUS,0.024559,0.000476,0.028396,0.000227
BNDX,0.000210,0.000866,0.000227,0.000988


In [40]:
ledoiid_wolf_sf = risk_models.risk_matrix(prices, method='ledoit_wolf_single_factor', frequency=252)
ledoiid_wolf_sf

,VOO,BND,VXUS,BNDX
VOO,0.028758,0.000164,0.024899,0.000184
BND,0.000164,0.001967,0.000442,0.000876
VXUS,0.024899,0.000442,0.028383,0.000200
BNDX,0.000184,0.000876,0.000200,0.000988


In [41]:
ledoiid_wolf_cv = risk_models.risk_matrix(prices, method='ledoit_wolf_constant_variance', frequency=252)
ledoiid_wolf_cv

,VOO,BND,VXUS,BNDX
VOO,0.028532,0.000161,0.024489,0.000181
BND,0.000161,0.002182,0.000435,0.000862
VXUS,0.024489,0.000435,0.028163,0.000197
BNDX,0.000181,0.000862,0.000197,0.001219


In [42]:
oracle_approx = risk_models.risk_matrix(prices, method='oracle_approximating', frequency=252)
oracle_approx

,VOO,BND,VXUS,BNDX
VOO,0.028737,0.000163,0.024861,0.000184
BND,0.000163,0.001987,0.000442,0.000875
VXUS,0.024861,0.000442,0.028363,0.000200
BNDX,0.000184,0.000875,0.000200,0.001009


In [43]:
expected_return = mhr
cov_matrix = oracle_approx

In [44]:
min_ret = max(0, min(expected_return))
max_ret = min(0.25, max(expected_return))
rets_seq = list(np.arange(min_ret, max_ret, 0.005))

weights = {}
max_sr = -1
max_r = 0.0
for r in rets_seq:
    print("****** {0} ******".format(r))
    _ef_bl = efficient_frontier.EfficientFrontier(expected_return, cov_matrix)
    _ef_bl.add_objective(objective_functions.L2_reg, gamma=1.0)
    _w = _ef_bl.efficient_return(r)
    print(_w)
    _perf = _ef_bl.portfolio_performance(verbose=False, risk_free_rate=0.005)
    print(_perf)
    weights[r] = (_perf, _w)
    
    if _perf[2] > max_sr:
        max_sr = _perf[2]
        max_r = r
print("{0}:{1}".format(max_r, max_sr))

****** 0.02496316149159461 ******
OrderedDict([('VOO', 0.2437823821626061), ('BND', 0.2560575765128826), ('VXUS', 0.2437981342998963), ('BNDX', 0.2563619070246148)])
(0.060865687365708784, 0.08236387832226097, 0.6782789798596655)
****** 0.02996316149159461 ******
OrderedDict([('VOO', 0.2437823821626061), ('BND', 0.2560575765128826), ('VXUS', 0.2437981342998963), ('BNDX', 0.2563619070246148)])
(0.060865687365708784, 0.08236387832226097, 0.6782789798596655)
****** 0.03496316149159461 ******
OrderedDict([('VOO', 0.2437823821626061), ('BND', 0.2560575765128826), ('VXUS', 0.2437981342998963), ('BNDX', 0.2563619070246148)])
(0.060865687365708784, 0.08236387832226097, 0.6782789798596655)
****** 0.03996316149159461 ******
OrderedDict([('VOO', 0.2437823821626061), ('BND', 0.2560575765128826), ('VXUS', 0.2437981342998963), ('BNDX', 0.2563619070246148)])
(0.060865687365708784, 0.08236387832226097, 0.6782789798596655)
****** 0.044963161491594614 ******
OrderedDict([('VOO', 0.2437823821626061), ('B

In [45]:
weights[max_r]

((0.13496316149159465, 0.16922146940585658, 0.768006340731472),
 OrderedDict([('VOO', 0.9867326416306637),
              ('BND', 0.0),
              ('VXUS', 0.013267358369336),
              ('BNDX', 0.0)]))

In [46]:
from sklearn.model_selection import TimeSeriesSplit

In [52]:
tscv = TimeSeriesSplit(n_splits=3, max_train_size=252*5, test_size=252, gap=0)
for train, test in tscv.split(prices):
    print(prices.iloc[train, :])
    print("\n")

                   VOO        BND       VXUS       BNDX
Date                                                   
2014-02-24  146.295654  65.757629  41.434612  41.827778
2014-02-25  146.243851  65.887291  41.379135  41.919296
2014-02-26  146.243851  66.025085  41.173119  41.977512
2014-02-27  147.055725  66.106155  41.371208  42.069027
2014-02-28  147.384033  66.057495  41.458370  42.019119
...                ...        ...        ...        ...
2019-02-19  242.375488  74.460449  47.052628  50.658855
2019-02-20  242.935379  74.432518  47.272663  50.677299
2019-02-21  242.052841  74.283592  47.125977  50.668076
2019-02-22  243.504715  74.451118  47.419361  50.732605
2019-02-25  243.865311  74.423203  47.630234  50.704956

[1260 rows x 4 columns]


                   VOO        BND       VXUS       BNDX
Date                                                   
2015-02-24  170.161041  69.309334  41.927937  45.417873
2015-02-25  169.977127  69.375999  42.017929  45.519310
2015-02-26  169.78447